### Installs

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 30.3MB/s 
     |████████████████████████████████| 901kB 41.6MB/s 
     |████████████████████████████████| 3.3MB 41.6MB/s 


### Imports

In [ ]:
# import HuggingFace models
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# import csv to deal with .csv data files
import csv

# import pandas to just be able to visualise our data files
import pandas as pd

# classic shit
import numpy as np
import torch

# for getting right format for data
from torch.utils.data import Dataset, DataLoader

# for computing metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

### Model and Tokenizer setup

In [ ]:
MODEL_NAME = "cardiffnlp/twitter-roberta-base" 

# define model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2, output_attentions=False, output_hidden_states=False)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side = "right")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and

# 1. Data

# Twitter racism parsed dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_set = pd.read_csv("twitter_racism_parsed_dataset.csv")

In [ ]:
data_set.shape

(13471, 5)

In [ ]:
data_set.head()

,index,id,Text,Annotation,oh_label
0,5.770000e+17,5.770000e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.410000e+17,5.410000e+17,#NULL!,none,0
2,5.680000e+17,5.680000e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.770000e+17,5.770000e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.710000e+17,5.710000e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [ ]:
data_set.drop(columns=['index','id', 'Annotation'], inplace=True)

In [ ]:
data_set.head()

,Text,oh_label
0,@AAlwuhaib1977 Muslim mob violence against Hin...,1
1,#NULL!,0
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,1
3,"Finally I'm all caught up, and that sudden dea...",0
4,@carolinesinders @herecomesfran *hugs*,0


In [ ]:
# Drop rows with empty text
data_set.drop( data_set[data_set.Text.str.len() < 7].index, inplace=True)

In [ ]:
data_set.head()

,Text,oh_label
0,@AAlwuhaib1977 Muslim mob violence against Hin...,1
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,1
3,"Finally I'm all caught up, and that sudden dea...",0
4,@carolinesinders @herecomesfran *hugs*,0
5,"Please, PLEASE start using ""is your discernmen...",0


In [ ]:
#!pip install emoji

In [ ]:
data_set.shape

(13467, 2)

In [ ]:
def normalise_text (text):
    text = text.str.lower() # lowercase
    text = text.str.replace(r"\#","") # replaces hashtags
    text = text.str.replace(r"http\S+","URL")  # remove URL addresses
    text = text.str.replace(r"@","")
    text = text.str.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.str.replace("\s{2,}", " ")
    return text

In [ ]:
data_set["Text"]=normalise_text(data_set["Text"])

In [ ]:
data_set.head(100)

,Text,oh_label
0,aalwuhaib1977 muslim mob violence against hind...,1
2,jncatron isra jourisra ampalestine islamophobi...,1
3,finally i'm all caught up and that sudden deat...,0
4,carolinesinders herecomesfran hugs,0
5,"please please start using ""is your discernment...",0
...,...,...
96,raniakhalek maxblumenthal nickkristof muslim b...,1
97,westcoastanime yes i found it ),0
98,someone said they were not going to attend sup...,0
99,dirtytruths certainly the paper shows that th...,0


In [ ]:
data_set.shape

(13467, 2)

In [ ]:
data_set2 = data_set

In [ ]:
data_set2.shape

(13467, 2)

In [ ]:
data_set2

,Text,oh_label
0,@AAlwuhaib1977 Muslim mob violence against Hin...,1
2,@jncatron @isra_jourisra @AMPalestine Islamoph...,1
3,"Finally I'm all caught up, and that sudden dea...",0
4,@carolinesinders @herecomesfran *hugs*,0
5,"Please, PLEASE start using ""is your discernmen...",0
...,...,...
13466,#mkr NOOOOO!!! I wanted Kat and Andre to lose!!!,0
13467,RT @MumtazCeltik: @WhiteHouse @VP \n\n#Kobane ...,0
13468,Glad the proper competition is starting and ev...,0
13469,"No worries Pete and Manu, take all the time of...",0


In [ ]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

import emoji

import string
import re

In [ ]:
def clean_tweet(tweet):
    
    # tokenize tweet
    tokenized_tweet = " ".join(tokenizer.tokenize(tweet))
    # remove unprintable characters (ð\x9f\x98\x81ð\x9f\x98\x81 for ex)
    printable_tweet = ''.join(filter(lambda x: x in set(string.printable), tokenized_tweet))
    
    # replace links (https://www. for ex) HTTPURL
    linkless_tweet = re.sub(r"http\S+", "HTTPURL", printable_tweet)
    
    # remove retweets (RT)
    retweetless_tweet = re.sub(r"RT", "", linkless_tweet)
    
    # replace mentions (@user for example) with @USER
    mentionless_tweet = re.sub(r"@[\S]+", "@USER", retweetless_tweet)
    
    # remove other odd stuff (&... for ex)
    other = re.sub(r"&[\S]+", "", mentionless_tweet)
    
    # replace emojis with str
    emojiless_tweet = emoji.demojize(other)
    
    # remove unwanted whitespace (ex "Hello        XWorld" => "hell world")
    whitespaceless = re.sub(r'\s+', ' ', emojiless_tweet)
    
    return whitespaceless

In [ ]:
#train_data_tweets = [clean_tweet(row["text"]) for row in train_data]
#train_data_labels = [int(row["label"]) for row in train_data]
text_rows = []

In [ ]:

for row in data_set2['Text']:
  text_rows.append(row)

In [ ]:
len(text_rows)

13467

In [ ]:
text_rows[1:10]

['@jncatron @isra_jourisra @AMPalestine Islamophobia is like the idea of Naziphobia. Islam is a religion of hate and it must be outlawed.',
 "Finally I'm all caught up, and that sudden death cook off looks like it's gonna be intense #MKR",
 '@carolinesinders @herecomesfran *hugs*',
 'Please, PLEASE start using "is your discernment blunted by steroids" to mean "are you on DRUGS?" from now on. DEAD',
 '@aymannathem As soon as ISIS chased all the minorities out of Mosul, the Sunni Arabs were happy to steal their property.  So fuck them.',
 '@Ali_Gharib @MaxBlumenthal Glad you like it. http://t.co/3ME3Nrk8xZ',
 '@HuffPostRelig Islam invaded and conquered 2/3 of Christiandom before any Christian crusades in response. The writer is a liar.',
 '@semzyxx Do you approve of your pedophile prophet raping a 9 year old girl, like it says in 7 hadith?',
 '@watan71969 @geeky_zekey Problem with vile Muslims is that they try to rationalize &amp; excuse the crimes of Islam rather than get rid of them.']

In [ ]:
train_data_tweets = [clean_tweet(row) for row in text_rows]

In [ ]:
train_data_tweets[1:10]

['@USER @USER @USER Islamophobia is like the idea of Naziphobia . Islam is a religion of hate and it must be outlawed .',
 "Finally I'm all caught up , and that sudden death cook off looks like it's gonna be intense #MKR",
 '@USER @USER * hugs *',
 'Please , PLEASE start using " is your discernment blunted by steroids " to mean " are you on DRUGS ? " from now on . DEAD',
 '@USER As soon as ISIS chased all the minorities out of Mosul , the Sunni Arabs were happy to steal their property . So fuck them .',
 '@USER @USER Glad you like it . HTTPURL',
 '@USER Islam invaded and conquered 2/3 of Christiandom before any Christian crusades in response . The writer is a liar .',
 '@USER Do you approve of your pedophile prophet raping a 9 year old girl , like it says in 7 hadith ?',
 '@USER @USER Problem with vile Muslims is that they try to rationalize & excuse the crimes of Islam rather than get rid of them .']

In [ ]:
label_rows = []
for row in data_set2['oh_label']:
  label_rows.append(row)

In [ ]:
train_data_labels = [row for row in label_rows]

In [ ]:
len(train_data_labels)

13467

In [ ]:
train_data_labels[1:10]

[1, 0, 0, 0, 0, 0, 1, 1, 1]

In [ ]:
import pandas as pd
  
# list of strings
lst1 = train_data_tweets
  
# Calling DataFrame constructor on list
# with indices and columns specified
df1 = pd.DataFrame(lst1, columns =['Text'])

In [ ]:
df1

,Text
0,@USER Muslim mob violence against Hindus in Ba...
1,@USER @USER @USER Islamophobia is like the ide...
2,"Finally I'm all caught up , and that sudden de..."
3,@USER @USER * hugs *
4,"Please , PLEASE start using "" is your discernm..."
...,...
13462,#mkr NOOOOO ! ! ! I wanted Kat and Andre to lo...
13463,@USER : @USER @USER #Kobane #JeSuisCharlie ' ...
13464,Glad the proper competition is starting and ev...
13465,"No worries Pete and Manu , take all the time o..."


In [ ]:
# list of strings
lst2 = train_data_labels

# Calling DataFrame constructor on list
# with indices and columns specified
df2 = pd.DataFrame(lst2, columns =['oh_label'])

In [ ]:
df2

,oh_label
0,1
1,1
2,0
3,0
4,0
...,...
13462,0
13463,0
13464,0
13465,0


In [ ]:
result1 = pd.concat([df1, df2], axis=1)

In [ ]:
result1

,Text,oh_label
0,@USER Muslim mob violence against Hindus in Ba...,1
1,@USER @USER @USER Islamophobia is like the ide...,1
2,"Finally I'm all caught up , and that sudden de...",0
3,@USER @USER * hugs *,0
4,"Please , PLEASE start using "" is your discernm...",0
...,...,...
13462,#mkr NOOOOO ! ! ! I wanted Kat and Andre to lo...,0
13463,@USER : @USER @USER #Kobane #JeSuisCharlie ' ...,0
13464,Glad the proper competition is starting and ev...,0
13465,"No worries Pete and Manu , take all the time o...",0


In [ ]:
test_set_df = result1[11500:]
test_set_df

,Text,oh_label
11500,""" Speaking up "" isn't the same thing as "" spea...",0
11501,rofl . hey guys . guys . GUYS . I've got somet...,0
11502,Fashion by Randi .,0
11503,@USER I think the paperwhite would be a better...,0
11504,@USER : #Kurdistan : #IS fighters threaten to...,0
...,...,...
13462,#mkr NOOOOO ! ! ! I wanted Kat and Andre to lo...,0
13463,@USER : @USER @USER #Kobane #JeSuisCharlie ' ...,0
13464,Glad the proper competition is starting and ev...,0
13465,"No worries Pete and Manu , take all the time o...",0


In [ ]:
len(test_set_df)

1967

In [ ]:
data_set=  result1.to_dict('r')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [ ]:
text_data = [row["Text"] for row in data_set]
label_data = [row["oh_label"] for row in data_set]


In [ ]:
print(len(text_data))
print(len(label_data))

13467
13467


In [ ]:
train_texts = text_data[:11500]
train_labels = label_data[:11500]

test_texts = text_data[11500:]
test_labels = label_data[11500:]

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_encodings[0]

Encoding(num_tokens=65, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
class TwitterRacismDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = TwitterRacismDataset(train_encodings, train_labels)
val_dataset = TwitterRacismDataset(val_encodings, val_labels)
test_dataset = TwitterRacismDataset(test_encodings, test_labels)

# 2. Training

### Set various parameters

Model hyperparameters

In [ ]:
# Hyperparameters for training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=30, 
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    load_best_model_at_end = True, 
    learning_rate=0.001,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    # evaluation_strategy='steps',
)

Metrics

In [ ]:
# A function computing metrics based on model output
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

### Train the model

Set model to training mode

In [ ]:
model.train()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

Define `Trainer` (training setup)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

Only train classifier layer. This code freezes all BERT layers for training.

In [ ]:
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

Finally train the model

In [ ]:
trainer.train()

Step,Training Loss
10,0.676300
20,0.509200
30,0.467700
40,0.427000
50,0.314300
60,0.375200
70,0.253300
80,0.202900
90,0.298000
100,0.292400


TrainOutput(global_step=17250, training_loss=0.18542990911870763, metrics={'train_runtime': 1290.72, 'train_samples_per_second': 13.365, 'total_flos': 1.34170213788e+16, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 2398523392, 'init_mem_gpu_alloc_delta': 499887104, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 13684736, 'train_mem_gpu_alloc_delta': 7125504, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 70300672})

### Evaluate on dev set

In [ ]:
trainer.evaluate()

{'epoch': 30.0,
 'eval_accuracy': 0.931304347826087,
 'eval_f1': 0.7669616519174042,
 'eval_loss': 0.15576007962226868,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 89817600,
 'eval_precision': 0.7323943661971831,
 'eval_recall': 0.804953560371517,
 'eval_runtime': 6.6272,
 'eval_samples_per_second': 347.057}

# 3. Testing

In this part we test our model on our test set.

Set the model to evaluate mode.

In [ ]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

Function to predict the class of an input.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_sent_pred(input_str,device=device):
    tok = tokenizer(input_str, return_tensors="pt", truncation=True, padding=True)
    tok.to(device)
    with torch.no_grad():
        pred = model(**tok)
    return pred['logits'].argmax(-1).item()

Function to compute metrics of model output for test data

In [ ]:
def compute_metrics_test(y_true,y_pred):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    acc = accuracy_score(y_true, y_pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
}

Compute metrics

In [ ]:
compute_metrics_test(test_labels, [get_sent_pred(sent) for sent in test_texts])

{'accuracy': 0.922216573462125,
 'f1': 0.7552,
 'precision': 0.7065868263473054,
 'recall': 0.8109965635738832}